In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import jax
jax.config.update('jax_platform_name', 'cpu')

In [3]:
from entot.data.data import create_gaussians, create_gaussian_split
from entot.plotting.plots import plot_1D
from entot.models.models import NoiseOutsourcingModel, KantorovichGapModel
from entot.models.utils import DataLoader
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm

import numpy as np
import jax
import jax.numpy as jnp

In [4]:
def h5py_to_dataset(path, img_size=64, batch_size: int = 64):
    with h5py.File(path, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[0]

        # Get the data
        data = list(f[a_group_key])
    
        #dataset = 2 * jnp.transpose(jnp.array(data) / 255., (0, 3, 1, 2))- 1
        dataset = jnp.array(data) / 255.
        dataset = jax.image.resize(dataset, (len(dataset), img_size, img_size, 3), method='bilinear')    

    return DataLoader(dataset, batch_size=batch_size)

In [5]:
handbags = h5py_to_dataset("../../../../data/handbag_128.hdf5")

Keys: <KeysViewHDF5 ['imgs']>


In [6]:
shoes = h5py_to_dataset("../../../../data/shoes_128.hdf5")

Keys: <KeysViewHDF5 ['imgs']>


In [8]:
from entot.nets import UNet, UResNet
import jax
import optax

In [9]:
unet = UNet()

In [10]:
state_unet = unet.create_train_state(jax.random.PRNGKey(0), optax.adam(1e-3), (64, 64, 3))

In [12]:
unet = UResNet()

In [13]:
state_unet = unet.create_train_state(jax.random.PRNGKey(0), optax.adam(1e-3), (64, 64, 3))

In [12]:
def train_step():
    def loss(params, state_unet, batch):
        predicted = state_unet.apply_fn({"params": params}, batch)
        print(predicted.shape)
        return jnp.mean((predicted.flatten() - batch.flatten())**2)


    def step_fn(state_unet, batch):
        grad_fn = jax.value_and_grad(loss, argnums=0)
        obj_value, grads = grad_fn(state_unet.params, state_unet, batch)
        return state_unet.apply_gradients(grads=grads), obj_value

    return step_fn

In [13]:
ts = train_step()
values = []

for step in range(1):
    batch = handbags(jax.random.PRNGKey(0))
    state_unet, obj_value = ts(state_unet, batch)
    values.append(obj_value)

(64, 64, 64, 3)


In [10]:
def h5py_2(path, img_size=64, batch_size: int = 64):
    with h5py.File(path, "r") as f:
        # List all groups
        print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[0]

        # Get the data
        data = list(f[a_group_key])
    
        #dataset = 2 * jnp.transpose(jnp.array(data) / 255., (0, 3, 1, 2))- 1
        #dataset = jnp.array(data) / 255.
        #dataset = jax.image.resize(dataset, (len(dataset), img_size, img_size, 3), method='bilinear')    

    return jnp.array(data)

In [51]:
h = h5py_2("../../../../data/handbag_128.hdf5")

Keys: <KeysViewHDF5 ['imgs']>


In [53]:
h.shape

(138767, 128, 128, 3)

In [ ]:
kg = KantorovichGapModel(epsilon=0.1, input_dim=10, iterations=5000)

In [15]:
from entot.nets import UNet

ImportError: cannot import name 'UNet' from 'entot.nets' (/home/dominikk/git_repos/entot/entot/nets/__init__.py)

In [12]:
dl = DataLoader(handbags, batch_size=32)

In [14]:
dir(entot.nets)

NameError: name 'entot' is not defined